In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import re
import os
import sklearn
import tqdm
from tqdm import tqdm
import nltk
from PIL import Image
import matplotlib.pyplot as plt
from skimage.transform import resize
import warnings
import skimage.io
import selective_search
warnings.filterwarnings("ignore") 

import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms

import cv2
import json

In [2]:
#### listing all jpeg files
def list_jpg_files(directory):
    jpg_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".jpg") or file.endswith(".jpeg"):
                jpg_files.append(os.path.join(root, file))
    return jpg_files

directory_path = "files"
jpg_files = list_jpg_files(directory_path)
print("JPEG files found:")
for jpg_file in jpg_files:
    print(jpg_file)

JPEG files found:
files/p11/p11000183/s50336039/3761aae0-255c0808-86d2121b-88ae172f-b7625d50.jpg
files/p11/p11000011/s51029426/ff213473-b64efa18-863f2bad-76181481-30bc30d7.jpg
files/p10/p10000032/s50414267/02aa804e-bde0afdd-112c0b34-7bc16630-4e384014.jpg
files/p10/p10000032/s50414267/174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962.jpg
files/p10/p10000764/s57375967/b79e55c3-735ce5ac-64412506-cdc9ea79-f1af521f.jpg
files/p10/p10000764/s57375967/096052b7-d256dc40-453a102b-fa7d01c6-1b22c6b4.jpg


In [3]:
chexpert = pd.read_csv("mimic-cxr-2.0.0-chexpert.csv")
chexpert = chexpert.fillna(0)

In [4]:
chexpert.head()

,subject_id,study_id,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,10000032,50414267,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,10000032,53189527,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,10000032,53911762,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,10000032,56699142,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,10000764,57375967,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0


In [5]:
chexpert['patient_id'] = chexpert['subject_id'].astype(str) + '_' + chexpert['study_id'].astype(str)
chexpert['diseases_label'] = chexpert.drop(['subject_id', 'study_id', 'patient_id'], axis=1).values.tolist()
chexpert = chexpert[['patient_id', 'diseases_label']]

In [6]:
chexpert["diseases_label"][0]

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [7]:
### get all regions
directory = 'scene_graph'

list
length_data = 0
list_regions = ["right lung", "right costophrenic angle","right cardiophrenic angle","right hemidiaphragm",
                "left lung", "left costophrenic angle","left hemidiaphragm","trachea","right clavicle",
               "left clavicle","aortic arch","mediastinum","cavoatrial junction","left cardiophrenic angle",
               "right atrium"]
dict_image = {}
dict_progress = {}
no_region_image_id = []
length_data_relation = 0
for filename in os.scandir(directory):
    name = filename.name.split("_")[0]
    if filename.path.endswith(".json"):
        for image_path in jpg_files:
            path_name = image_path.split("/")[-1].split(".")[0]
            if path_name == name :
                length_data = length_data +1
                f = open(filename)
                data = json.load(f)
                list_region = []
                count = len(data["objects"])
                if count > 15 :
                    for region in data["objects"]:
                        if region["bbox_name"] in list_regions:
                            list_region.append([region["original_x1"], region["original_y1"], region["original_x2"],region["original_y2"]])
                    dict_image[image_path] = list_region
                    progression_label = {"Better":0, "Worse" : 0, "Stable" :0}
                    if len(data["relationships"]) >= 1 :
                        for relation in data["relationships"]:
                            if relation["predicate"] == "Worse":
                                progression_label["Worse"] = 1
                            elif relation["predicate"] == "Better" :
                                progression_label["Better"] = 1
                            else :
                                progression_label["Stable"] = 1    
                    dict_progress[image_path] = list(progression_label.values())
           
                

In [8]:
for i in dict_progress:
    print(i)
    print(dict_progress[i])
    print("------")

files/p10/p10000032/s50414267/02aa804e-bde0afdd-112c0b34-7bc16630-4e384014.jpg
[0, 0, 0]
------
files/p10/p10000764/s57375967/096052b7-d256dc40-453a102b-fa7d01c6-1b22c6b4.jpg
[0, 0, 0]
------
files/p11/p11000011/s51029426/ff213473-b64efa18-863f2bad-76181481-30bc30d7.jpg
[0, 0, 0]
------
files/p11/p11000183/s50336039/3761aae0-255c0808-86d2121b-88ae172f-b7625d50.jpg
[0, 0, 1]
------


In [9]:
for i in dict_image:
    print(i)
    print(len(dict_image[i]))
    print("==================")
print(len(list_regions))

files/p10/p10000032/s50414267/02aa804e-bde0afdd-112c0b34-7bc16630-4e384014.jpg
15
files/p10/p10000764/s57375967/096052b7-d256dc40-453a102b-fa7d01c6-1b22c6b4.jpg
15
files/p11/p11000011/s51029426/ff213473-b64efa18-863f2bad-76181481-30bc30d7.jpg
15
files/p11/p11000183/s50336039/3761aae0-255c0808-86d2121b-88ae172f-b7625d50.jpg
15
15


In [10]:
#### stage1 prep
disease_labels = []
for i in dict_image:
    img_path = i.split("/")
    subject_id = img_path[2][1:]
    study_id = img_path[3][1:]
    tmp_str = subject_id+"_"+study_id
    for index, row in chexpert.iterrows():
        if tmp_str == row["patient_id"]:
            disease_labels.append(row["diseases_label"])

In [36]:
disease_labels

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
 [1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0]]

In [11]:
### cropping all images
def region_feature_preprocessing(image, bounding_box):

    cropped_image = image.crop(bounding_box)
    image_array = np.array(cropped_image)
    padded_image = image_array
    
    height, width = cropped_image.size 
    
    max_pad = max(height, width)
    if height < max_pad :
        pad_height = max_pad - height
        padded_image = np.pad(image_array, ((0, pad_height), (0,0),(0,0)), mode='constant')
        
    if width < max_pad :
        pad_width = max_pad - width
        padded_image = np.pad(image_array, ((0, 0), (0,pad_width), (0,0)), mode='constant')
    return Image.fromarray(padded_image)


#image_inputs = {}
#for image in dict_image:
#    aa = region_feature_preprocessing(image, dict_image[image])
#    image_inputs[image] = aa



In [17]:
len(image_inputs[0])

15

In [118]:
region_input_tensors = torch.tensor(image_inputs, dtype=torch.float32)

In [119]:
region_input_tensors.shape

torch.Size([4, 36, 1024])

In [12]:
### trying densenet121 on original image 224x224

model = models.densenet121(pretrained=True)

# Remove the original classifier (fully connected layers)
model_features = torch.nn.Sequential(*list(model.children())[:-1])

# Define global average pooling layer
global_avg_pooling = nn.AdaptiveAvgPool2d((1, 1))  # Perform global average pooling to (1, 1)


# Combine the layers in a Sequential module
model = nn.Sequential(model_features, global_avg_pooling, nn.Flatten())

# Set the model to evaluation mode
model.eval()

Sequential(
  (0): Sequential(
    (0): Sequential(
      (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu0): ReLU(inplace=True)
      (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (denseblock1): _DenseBlock(
        (denselayer1): _DenseLayer(
          (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (denselayer2): _DenseLayer(
          (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine

In [13]:
#### extracting the image features from the original image
def image_global_feature_extraction(image_path, model):
    transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to 224x224
    transforms.ToTensor(),  # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
    ])

    # Load and preprocess the input image
    image = Image.open(image_path)
    image = image.convert("RGB")
    print(type(image))
    image_tensor = transform(image).unsqueeze(0)
    extracted_features = model(image_tensor)
    return extracted_features

def image_local_feature_extraction(image_path, bounding_boxes, model):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize the image to 224x224
        transforms.ToTensor(),  # Convert the image to a PyTorch tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
    ])
    region_features = []    
    # Load and preprocess the input image
    image = Image.open(image_path)
    image = image.convert("RGB")
    for region in bounding_boxes:
        cropped_image = region_feature_preprocessing(image, region)  # Crop the image
        image_tensor = transform(cropped_image).unsqueeze(0)  # Apply transformations
        extracted_features = model(image_tensor)
        region_features.append(extracted_features)
    concat_region = torch.cat(region_features, dim=0)    
    return concat_region



full_image_features = []
for image in dict_image:
    global_features = image_global_feature_extraction(image,model)
    local_features = image_local_feature_extraction(image,dict_image[image], model)
    combined_features = torch.cat((global_features, local_features), dim=0)
    full_image_features.append(combined_features)

input_features = torch.stack(full_image_features, dim=0)
input_features.shape


<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>


torch.Size([4, 16, 1024])

In [15]:
target = torch.tensor(disease_labels)


In [59]:
import torch
import torch.nn as nn

class TransformerDecoder(nn.Module):
    def __init__(self, input_size, output_size, num_layers, num_heads, hidden_size, dropout=0.1):
        super(TransformerDecoder, self).__init__()
        self.transformer_decoder = nn.TransformerDecoder(
            decoder_layer=nn.TransformerDecoderLayer(d_model=input_size, nhead=num_heads),
            num_layers=num_layers
        )
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, tgt, memory):
        # tgt shape: [batch_size, tgt_seq_len, input_size]
        # memory shape: [batch_size, src_seq_len, input_size]
        transformer_output = self.transformer_decoder(tgt, memory)
        output = self.fc(transformer_output[:, -1, :])  # Reduce the sequence length dimension
        return output

# Example usage
input_size = 1024
output_size = 14
num_layers = 6
num_heads = 8
hidden_size = 2048

# Instantiate the transformer decoder model
model = TransformerDecoder(input_size, output_size, num_layers, num_heads, hidden_size)

# Example input tensors
tgt_tensor = torch.randn(4, 16, 1024)  # Target sequence
memory_tensor = torch.randn(4, 16, 1024)  # Encoder output (memory)

# Forward pass
output_tensor = model(tgt_tensor, memory_tensor)

print("Output tensor size:", output_tensor.size())  # Expected output size: [4, 14]


Output tensor size: torch.Size([4, 14])


In [48]:
tgt.shape

torch.Size([4, 14])

In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

# Decoder Block
class DecoderBlock(nn.Module):
    def __init__(self, d_model, num_heads, ff_hidden_layer, dropout):
        super(DecoderBlock, self).__init__()

        self.self_attention = nn.MultiheadAttention(d_model, num_heads, dropout=dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.linear1 = nn.Linear(d_model, ff_hidden_layer)
        self.linear2 = nn.Linear(ff_hidden_layer, d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout2 = nn.Dropout(dropout)

    
    def forward(self, x, target_mask):
        attn_output, _ = self.self_attention(x, x, x, attn_mask=target_mask)
        x = x + self.dropout1(attn_output)
        x = self.norm1(x)
        ff_output = self.linear2(F.relu(self.linear1(x)))
        x = x + self.dropout2(ff_output)
        x = self.norm2(x)
        return x

# Positional Encoding
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer("pe", pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

# Transformer Decoder
class TransformerDecoder(nn.Module):
    def __init__(self, input_size, output_size, d_model, num_heads, ff_hidden_layer, dropout):
        super(TransformerDecoder, self).__init__()

        self.embedding = nn.Linear(input_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        self.transformer_block = DecoderBlock(d_model, num_heads, ff_hidden_layer, dropout)
        self.linear = nn.Linear(d_model, 32)
        self.linear2 = nn.Linear(d_model, output_size)
        self.softmax = nn.LogSoftmax(dim=-1)

    
    def forward(self, x):
        x = self.embedding(x)
        x = self.pos_encoder(x)
        tgt_mask = generate_square_subsequent_mask(x.size(0))
        x = self.transformer_block(x, tgt_mask)
        output = self.linear(x)
        output2 = self.linear2(output.view(output.size(0),-1))
        output = self.softmax(output2)
        return output

# Function to generate a square subsequent mask
def generate_square_subsequent_mask(sz):
    """Generate a mask to prevent attention to future positions."""
    mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

# Model parameters
input_size = 1024  # Dimensionality of input
output_size = 14   # Dimensionality of output
d_model = 512      # Dimensionality of the model
num_heads = 8      # Number of attention heads
ff_hidden_layer = 2048  # Hidden layer size in feedforward network within transformer block
dropout = 0.1      # Dropout rate
num_layers = 6     # Number of decoder layers

# Initialize the model
model = TransformerDecoder(input_size, output_size, d_model, num_heads, ff_hidden_layer, dropout)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# Create a tensor representing batch size, sequence length, and input size
input_tensor = torch.randn(4, 16, 1024)
for epoch in range(10):  # Training for 100 epochs
    model.train()  # Set the model to training mode

    optimizer.zero_grad()
    output = model(input_tensor)  # Forward pass
    print(output.shape)
    # Calculate loss with attention
    loss = criterion(output, target)
    # Backpropagation
    loss.backward()
    optimizer.step()

    # Print loss
    
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 1000, loss.item()))
# Forward pass through the model



torch.Size([4, 14])
Epoch [1/1000], Loss: 5.7957
torch.Size([4, 14])
Epoch [2/1000], Loss: 2.2883
torch.Size([4, 14])
Epoch [3/1000], Loss: 1.1697
torch.Size([4, 14])
Epoch [4/1000], Loss: -0.2263
torch.Size([4, 14])
Epoch [5/1000], Loss: -2.5283
torch.Size([4, 14])
Epoch [6/1000], Loss: -4.7633
torch.Size([4, 14])
Epoch [7/1000], Loss: -6.6540
torch.Size([4, 14])
Epoch [8/1000], Loss: -6.9325
torch.Size([4, 14])
Epoch [9/1000], Loss: -7.6276
torch.Size([4, 14])
Epoch [10/1000], Loss: -9.0170


In [18]:
output.shape

torch.Size([4, 14])

In [58]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import torch.nn as nn
import torch.nn.functional as F

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads, dropout=0.1):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads

        self.query = nn.Linear(d_model, d_model)
        self.key = nn.Linear(d_model, d_model)
        self.value = nn.Linear(d_model, d_model)

        self.fc_out = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim]))

    def forward(self, query, key, value, mask=None):
        batch_size = query.shape[0]

        # Perform linear transformation and split into multiple heads
        Q = self.query(query).view(batch_size, -1, self.num_heads, self.head_dim).permute(0, 2, 1, 3)
        K = self.key(key).view(batch_size, -1, self.num_heads, self.head_dim).permute(0, 2, 1, 3)
        V = self.value(value).view(batch_size, -1, self.num_heads, self.head_dim).permute(0, 2, 1, 3)

        # Compute scaled dot-product attention
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale
        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)

        attention = torch.softmax(energy, dim=-1)
        x = torch.matmul(self.dropout(attention), V)

        # Reshape and concatenate heads
        x = x.permute(0, 2, 1, 3).contiguous().view(batch_size, -1, self.d_model)

        # Apply final linear layer
        x = self.fc_out(x)

        return x

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=0.1)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-torch.log(torch.tensor(10000.0)) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)


class DecoderLayer(nn.Module):
    def __init__(self, h, d_model, d_k, d_v, d_ff, rate):
        super(DecoderLayer, self).__init__()
        self.multihead_attention1 = MultiHeadAttention(d_model, h, rate)
        self.dropout1 = nn.Dropout(rate)
        self.add_norm1 = AddNorm(d_model)
        self.multihead_attention2 = MultiHeadAttention(d_model, h, rate)
        self.dropout2 = nn.Dropout(rate)
        self.add_norm2 = AddNorm(d_model)
        self.feed_forward = FeedForward(d_model, d_ff)
        self.dropout3 = nn.Dropout(rate)
        self.add_norm3 = AddNorm(d_model)

    def forward(self, x, encoder_output, look_ahead_mask, padding_mask):
        # Multi-head attention layer
        multihead_output1 = self.multihead_attention1(x, x, x, look_ahead_mask)

        # Add in a dropout layer
        multihead_output1 = self.dropout1(multihead_output1)

        # Followed by an Add & Norm layer
        add_norm_output1 = self.add_norm1(x, multihead_output1)

        # Followed by another multi-head attention layer
        multihead_output2 = self.multihead_attention2(add_norm_output1, encoder_output, encoder_output, padding_mask)

        # Add in another dropout layer
        multihead_output2 = self.dropout2(multihead_output2)

        # Followed by another Add & Norm layer
        add_norm_output2 = self.add_norm2(add_norm_output1, multihead_output2)

        # Followed by a fully connected layer
        feedforward_output = self.feed_forward(add_norm_output2)

        # Add in another dropout layer
        feedforward_output = self.dropout3(feedforward_output)

        # Followed by another Add & Norm layer
        return self.add_norm3(add_norm_output2, feedforward_output)



class Decoder(nn.Module):
    def __init__(self, output_size, max_seq_length, num_layers, h, d_k, d_v, d_model, d_ff, rate):
        super(Decoder, self).__init__()
        self.pos_encoding = PositionalEncoding(max_seq_length, d_model)
        self.dropout = nn.Dropout(rate)
        self.decoder_layers = nn.ModuleList([DecoderLayer(h, d_k, d_v, d_model, d_ff, rate) for _ in range(num_layers)])
        self.fc = nn.Linear(d_model, output_size)

    def forward(self, output_target, encoder_output, look_ahead_mask, padding_mask):
        # Generate the positional encoding
        pos_encoding_output = self.pos_encoding(output_target)

        # Add in a dropout layer
        x = self.dropout(pos_encoding_output)

        # Pass on the positional encoded values to each decoder layer
        for decoder_layer in self.decoder_layers:
            x = decoder_layer(x, encoder_output, look_ahead_mask, padding_mask)

        # Apply a fully connected layer
        output = self.fc(x)

        return output


# Usage example
output_size = 14  # Output size for the decoder
max_seq_length = 16  # Maximum length of the output sequence
num_layers = 6  # Number of layers in the decoder stack
h = 8  # Number of self-attention heads
d_k = 64  # Dimensionality of the linearly projected keys
d_v = 64  # Dimensionality of the linearly projected values
d_ff = 2048  # Dimensionality of the inner fully connected layer
d_model = 1024  # Dimensionality of the model sub-layers' outputs
rate = 0.1  # Dropout rate

batch_size = 4  # Batch size from the training process

input_seq = torch.randn(batch_size, max_seq_length, d_model)
encoder_output = torch.randn(batch_size, max_seq_length, d_model)
look_ahead_mask = None  # You need to define this mask
padding_mask = None  # You need to define this mask

decoder = Decoder(output_size, max_seq_length, num_layers, h, d_k, d_v, d_model, d_ff, rate)
print(decoder(input_seq, encoder_output, look_ahead_mask, padding_mask))


NameError: name 'AddNorm' is not defined